<a href="https://colab.research.google.com/github/elhamod/IS813/blob/main/Week6/IS883_2024_Week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IS883 Week 10: RAG, ReAct, and CoT


1. Use Google Colab for this assignment.

2. **You are allowed to use ChatGPT for this assignment (except where specifically instructed). However, you may use Google and other online resources. As per the syllabus, you are required to cite your usage. You are also responsible for understanding the solution and defending it when asked in class.**

3. For each question, fill in the answer in the cell(s) right below it. The answer could be code or text. You can add as many cells as you need for clarity.

4. Enter your BUID (only numerical part) below, if applicable.

5. **Your submission on Blackboard should be the downloaded notebook (i.e., ipynb file). It should be prepopulated with your solution (i.e., the TA and/or instructor need not rerun the notebook to inspect the output). The code, when executed by the TA and/or instructor, should run with no runtime errors.**

#Part 1: Pre-class Work

In [1]:
!pip install langchain_google_community openai tiktoken faiss-cpu pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294

# Part 2: In-class Work

## 2.1 RAG: Question Answering System Using the School's Syllabus Database

 At your school, the department has embarked on a project to utilize language modeling for the development of a question-answering agent. This initiative aims to streamline the access to information for faculty and staff, particularly regarding the extensive array of courses offered at our institution. The data pertaining to these courses is currently dispersed across numerous documents within [the department's syllabus corpus](https://drive.google.com/drive/folders/1dH-t_Ujih4lMMzUOaNOHngvOYLK_gWOp?usp=sharing).

Download the corpus to your Google Drive and update the path below.

Note: The used syllabus corpus is a subset of [Cal Poly's Syllabus Corpus dataset](https://www.kaggle.com/datasets/mfekadu/syllabus-corpus).

In [2]:
from google.colab import drive
drive.mount('/content/drive')

syllabus_corpus_path = "/content/drive/MyDrive/IS883/Assignments/2023/IS883_HW4_syllabus_corpus"

Mounted at /content/drive


We use [PyPDFDirectoryLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.pdf.PyPDFDirectoryLoader.html) to create a loader that can load all the PDFs in the directory so they could be used by LangChain.

In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader(syllabus_corpus_path)

Given the extensive data contained within these documents, it's impractical to include them in their entirety in our queries. Including all data at once could exceed the context window's capacity and may result in significant processing costs. To address this challenge, we employ a methodical approach to manage the data effectively.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

Now, using the afortmentioned loader and splitter, perform the splitting.

In [5]:
chunks = loader.load_and_split(text_splitter)

The next crucial step involves the creation of a data store, essentially a database, that will house the chunks of data you've created. The effectiveness of our question-answering system hinges on its ability to swiftly locate the relevant chunk containing the answer to any given query. To achieve this efficiency, we will employ a sophisticated indexing strategy, rather than relying on a basic brute-force search method.

* Build the Data Store with [Facebook AI Similarity Search (FAISS)](https://python.langchain.com/docs/integrations/vectorstores/faiss): Set up your data store using a [FAISS Vector store](https://python.langchain.com/docs/integrations/vectorstores/faiss). FAISS is a library developed by Facebook AI that allows for efficient similarity search and clustering of dense vectors.

* Embedding Calculation with `OpenAIEmbeddings`: For each chunk of data in your store, calculate an embedding using `OpenAIEmbeddings`. These embeddings are essentially numerical representations of your text data, which can then be compared to the embeddings of incoming queries.

* Indexing for Efficient Search: By creating embeddings for each chunk and indexing them in the FAISS Vector store, you will enable the system to quickly find the most relevant chunk in response to a query. This process involves comparing the embedding of the query with the embeddings of the chunks to identify the best match.

The combination of `FAISS` and `OpenAIEmbeddings` will significantly enhance the efficiency and accuracy of the question-answering system, allowing for rapid retrieval of information from the extensive syllabus corpus.

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from google.colab import userdata
openai_api_key = userdata.get('MyOpenAIKey')

faiss_index = FAISS.from_documents(chunks, OpenAIEmbeddings(openai_api_key=openai_api_key))

<ipython-input-6-44c4f0c040a8>:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  faiss_index = FAISS.from_documents(chunks, OpenAIEmbeddings(openai_api_key=openai_api_key))


With the data store and indexing system in place, you are now equipped to tackle the core functionality of our question-answering system: responding to queries based on the indexed database.

* The code will utilize the [*`similarity_search`*](https://python.langchain.com/docs/integrations/vectorstores/faiss) function to identify the chunk that is most relevant or most similar to the posed question. This function will compare the embedding of the query with those of the indexed chunks to find the best match.

* Once you have identified the most relevant answer, output additional details indicating where this chunk is located. Specifically, provide information about *the page number and the document from which this chunk was extracted*.

To gain a deeper understanding of how similarity search operates, refer to the provided articles and references. These resources will offer a more detailed conceptual insight into the workings of similarity search algorithms and their applications in systems like ours.

[Resource 1.](https://www.pinecone.io/learn/what-is-similarity-search/)

[Resource 2.](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

In [7]:
question = "Who is the instructor of Linear Algebra III?"

In [10]:
relevant_chunks = faiss_index.similarity_search(question)
# print(relevant_chunks)
best_chunk = relevant_chunks[0]

print(str(best_chunk.metadata['source']) + ". Page: " + str(best_chunk.metadata['page']) + "\n\n", best_chunk.page_content)

/content/drive/MyDrive/IS883/Assignments/2023/IS883_HW4_syllabus_corpus/11___syllabus.pdf. Page: 0

 Math 406 – Linear Algebra III
Winter 2009
Course Syllabus
Instructor: Anton Kaul
Oﬃce: 25-312 (Faculty Oﬃces East)
Phone: 6-1678
email: akaul@calpoly.edu
Oﬃce Hours: Monday 2-4, Tuesday 9-10, Thursday 9-10
Course Web Page: www.calpoly.edu/ ∼akaul/teaching/Math406
Textbook
The text for the course is Friedberg, Insel, and Spence, Linear Algebra , 4th ed.
Course Description
In Math 406 we will continue our study of the fundamental concepts of Linear Algebra. Topics


Let's delve deeper into the results to evaluate the system.

Display the Top 5 Matches: Print the top five most relevant chunks in response to your query, *along with their respective similarity scores*. These scores quantify how closely each chunk matches your query, offering a clear metric of relevance.

In [25]:
answers_with_scores = faiss_index.similarity_search_with_score(question, k=5)

# Creating a DataFrame
import pandas as pd
df = pd.DataFrame(answers_with_scores, columns=['chunk', 'score'])

# Displaying the DataFrame
df

,chunk,score
0,page_content='Math 406 – Linear Algebra III\nW...,0.263624
1,page_content='Math 206 – Linear Algebra I\nSpr...,0.302393
2,page_content='Math 483/560 – Abstract Algebra ...,0.329265
3,page_content='Math 143 – Calculus III\nWinter ...,0.346784
4,page_content='Math 481 – Abstract Algebra I\nF...,0.349944


Finally, we are going to use OpenAI API to get the answer to the question based on the relevant chunk.

In [ ]:
temperature = 0.1  # A low temperature is used because we are interested in prcision more than creativity.

In [ ]:
from langchain.chat_models import ChatOpenAI

# Create a reference to the language model
llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=temperature, model_name="gpt-3.5-turbo-0613")

In [28]:
# Create a question answerting chain
from langchain.chains.question_answering import load_qa_chain
qa_chain = load_qa_chain(chat, verbose=True)

# #Use LangChain debugging
# import langchain
# langchain.debug = True

# Do question answering using the top matching chunk as an input document
relevant_chunks = faiss_index.similarity_search(question, k=5)
result = qa_chain.run(input_documents=relevant_chunks, question=question)
print("The question is:",question)
print("Th answer is:", result)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Math 406 – Linear Algebra III
Winter 2009
Course Syllabus
Instructor: Anton Kaul
Oﬃce: 25-312 (Faculty Oﬃces East)
Phone: 6-1678
email: akaul@calpoly.edu
Oﬃce Hours: Monday 2-4, Tuesday 9-10, Thursday 9-10
Course Web Page: www.calpoly.edu/ ∼akaul/teaching/Math406
Textbook
The text for the course is Friedberg, Insel, and Spence, Linear Algebra , 4th ed.
Course Description
In Math 406 we will continue our study of the fundamental concepts of Linear Algebra. Topics

Math 206 – Linear Algebra I
Spring 2020
Course Syllabus
Instructor: Anton Kaul
Oﬃce: 25-312 (Faculty Oﬃces East)
Phone: 756-1678
email: akaul@calpoly.edu
Oﬃce Hours: T 1-2:30, Th 1-2:30, via Zoom
Course Web Page: https://web.calpoly.edu/ ∼ak

## 2.2 Chain of Though: Riddle Me This...

Let's have some fun with math riddles and explore the impact of different prompt engineering frameworks on solving them using AI.

In [11]:
riddle  = "A man left 100 dollars to be divided between his two sons Alfred and Benjamin. If one third of Alfred’s legacy was taken from one-fourth of Benjamin’s, the remainder would be 11 dollars. How much is Alfred's legacy?"

In [12]:
from langchain.agents import AgentType, initialize_agent, load_tools

In [13]:
#Use LangChain debugging
import langchain
langchain.debug = False


**Solution 0: Zero-shot learning**.



In [14]:
## Create the llm
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")
print("\n", chat.invoke(riddle).content)

<ipython-input-14-e74a0985e670>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")



 Let A be the amount left to Alfred and B be the amount left to Benjamin.

A + B = 100 (from the total amount left by the man)

(1/3)A = (1/4)B + 11

Multiplying the second equation by 12 to eliminate the fractions:

4A = 3B + 132

Substitute the first equation into the second equation:

4(100 - B) = 3B + 132
400 - 4B = 3B + 132
400 - 132 = 3B + 4B
268 = 7B
B = 38

Substitute B back into the first equation:

A + 38 = 100
A = 100 - 38
A = 62

Therefore, Alfred's legacy is 62 dollars.


**Questions:**

1. What happens if we use a newer model?
2. What happens if we change temperature?
3. What happens if we use prompt engineering?

### **Solution 1: Zero-shot learning with a calculator**.


Let's try enhancing the zero-shot learning approach by integrating a calculator tool with the language model. This setup aims to improve the accuracy and effectiveness of solving riddles, especially those involving mathematical elements.

Use OpenAI playground with prompt engineering to trigger this.



### **Solution 2: Chain of Thought (CoT)**.


Finally, you'll guide the model through a step-by-step process, breaking down the solution into clear, logical steps. This CoT approach helps the AI model understand the reasoning process needed to arrive at the correct answer.


In [15]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "A woman left 200 dollars to be divided between her daughters. If one half of the Mary's inheritence was taken from one-quarter of the Sally's, the remainder would be 10 dollars. How much is Mary's inheritence?"
,
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: What's the sum equation?
Intermediate answer: 200 = x + y
Follow up: what's the fractions' equations?
Intermediate answer: 0.25*y - 0.5*x = 10
Follow up: get y in terms of x
Intermediate answer: y = (10 + 0.5*x) / 0.25
Follow up: substitute y into the first equation
Intermediate answer: 200 = x + (10 + 0.5*x) / 0.25
Follow up: solve for x
Intermediate answer: x + (0.5/0.25)x = 200 - 10/0.25
Follow up: simplify
Intermediate answer: x + 2x = 200 - 40
Follow up: Consolidate
Intermediate answer: 3x = 160
Follow up: get x
Intermediate answer: x = 160/3
The final answer: 53.333333
"""
  }
]

In [16]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n Answer: {answer} \n ---------")

In [17]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input=riddle))

Question: A woman left 200 dollars to be divided between her daughters. If one half of the Mary's inheritence was taken from one-quarter of the Sally's, the remainder would be 10 dollars. How much is Mary's inheritence?
 Answer: 
Are follow up questions needed here: Yes.
Follow up: What's the sum equation?
Intermediate answer: 200 = x + y
Follow up: what's the fractions' equations?
Intermediate answer: 0.25*y - 0.5*x = 10
Follow up: get y in terms of x
Intermediate answer: y = (10 + 0.5*x) / 0.25
Follow up: substitute y into the first equation
Intermediate answer: 200 = x + (10 + 0.5*x) / 0.25
Follow up: solve for x
Intermediate answer: x + (0.5/0.25)x = 200 - 10/0.25
Follow up: simplify
Intermediate answer: x + 2x = 200 - 40
Follow up: Consolidate
Intermediate answer: 3x = 160
Follow up: get x
Intermediate answer: x = 160/3
The final answer: 53.333333
 
 ---------

Question: A man left 100 dollars to be divided between his two sons Alfred and Benjamin. If one third of Alfred’s legacy 

Now, call the `llm` object with the template after properly substituting the riddle into it.

In [18]:
print(chat.invoke(prompt.format(input=riddle)).content)

Answer:
Are follow up questions needed here: Yes
Follow up: What's the sum equation?
Intermediate answer: 100 = x + y
Follow up: What are the fractions' equations?
Intermediate answer: 0.25*y - 0.33*x = 11
Follow up: Get y in terms of x
Intermediate answer: y = (11 + 0.33*x) / 0.25
Follow up: Substitute y into the first equation
Intermediate answer: 100 = x + (11 + 0.33*x) / 0.25
Follow up: Solve for x
Intermediate answer: x + (0.33/0.25)x = 100 - 11/0.25
Follow up: Simplify
Intermediate answer: x + 1.32x = 100 - 44
Follow up: Consolidate
Intermediate answer: 2.32x = 56
Follow up: Get x
Intermediate answer: x = 56 / 2.32
The final answer: 24.137931


## 2.3 ReAct

Let's see how we can use the wikipedia agent in `LangChain`

In [19]:
!pip install -U wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=bf166e3110b16263d260c9ec7349f1dd4a8d281765761a36653fa67323e49cea
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


Let's ask a question about GPT4 in Wikipedia

In [30]:
from langchain.agents import load_tools, initialize_agent
from langchain_community.utilities import SearchApiAPIWrapper
import os
os.environ["SEARCHAPI_API_KEY"] = userdata.get('SEARCHAPI_API_KEY')
SearchApiAPIWrapper()


chat = ChatOpenAI(openai_api_key=openai_api_key)
tools = load_tools(["wikipedia", "searchapi", "llm-math"], llm=chat)




agent= initialize_agent(
    tools,
    chat,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True)

# msg = "When was ChatGPT 4 created??"
msg = "When is Mohannad Elhamod's birthdate?"
# msg = "Which year was it when Mohannad Elhamod was 10?"

agent(msg)



> Entering new AgentExecutor chain...
I should search for Mohannad Elhamod's birthdate on Wikipedia.
Action: wikipedia
Action Input: Mohannad Elhamod
Observation: No good Wikipedia Search Result was found
Thought:I should try using the searchapi to find information about Mohannad Elhamod's birthdate.
Action: searchapi
Action Input: Mohannad Elhamod birthdate
Observation: Mohannad Elhamod (M'12) was born in Damascus, Syria, in 1984. He received the B.Sc. degree in electrical and computer engineering from Jordan University of ...
Mohannad Elhamod. PhD in Computer Science | Machine Learning | ex-SDE @Microsoft | ex-research intern @NASA. Questrom School of Business ...
Affiliations ; Mohannad Elhamod · Virginia Tech. ; Jie Bu · Virginia Tech. ; Christopher Singh · SUNY, Binghamton ; Matthew Redell · SUNY, Binghamton.
Mohannad Elhamod, Clinical Assistant Professor, Information Systems. Contact: Email: elhamod@bu.edu, opens in new tab. Download CV, opens in new tab.
Graduate Students of t

{'input': "When is Mohannad Elhamod's birthdate?",
 'output': 'Mohannad Elhamod was born in 1984.'}

Here is how you could list all tools available.

In [ ]:
langchain.agents.get_all_tool_names()

['sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-lens',
 'google-serper',
 'google-scholar',
 'google-finance',
 'google-trends',
 'google-jobs',
 'google-serper-results-json',
 'searchapi',
 'searchapi-results-json',
 'serpapi',
 'dalle-image-generator',
 'twilio',
 'searx-search',
 'merriam-webster',
 'wikipedia',
 'arxiv',
 'golden-query',
 'pubmed',
 'human',
 'awslambda',
 'stackexchange',
 'sceneXplain',
 'graphql',
 'openweathermap-api',
 'dataforseo-api-search',
 'dataforseo-api-search-json',
 'eleven_labs_text2speech',
 'google_cloud_texttospeech',
 'read_file',
 'reddit_search',
 'news-api',
 'tmdb-api',
 'podcast-api',
 'memorize',
 'llm-math',
 'open-meteo-api',
 'requests',
 'requests_get',
 'requests_post',
 'requests_patch',
 'requests_put',
 'requests_delete',
 'terminal']